### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os

### Get correct data

In [2]:
data_folder = './data'
data_format = 'xlsx'
print(f'All files in folder: {os.listdir(data_folder)}')
xls_files = []

for file in os.listdir(data_folder):
    if file.endswith(data_format):
        xls_files.append(file)
print(f'Filtred files: {xls_files}')

All files in folder: ['base_matches', 'elos', 'mensinglesatp.csv']
Filtred files: []


### Create dataframe for each file

In [3]:
dataframes = {}
for xls_file in xls_files:
    file_path = os.path.join(data_folder, xls_file)
    df = pd.read_excel(file_path, engine='openpyxl')
    dataframes[xls_file] = df
    
for df_name in dataframes:
    head = dataframes[df_name].head()
    print(f'{df_name}:\n {head}')

### Combine dataframes into single one

In [4]:
combined_df = pd.concat(dataframes.values(), axis=0)
combined_df = pd.read_csv('.\exports\concat_clean.csv')
combined_df.tail()

ValueError: No objects to concatenate

### Export dataframe into csv

In [5]:
combined_df.to_csv('./exports/combined_data_full.csv', index=True)

NameError: name 'combined_df' is not defined

### Drop unnecessary columns

In [6]:
# print(f'All columns:\n {combined_df.columns}')
# drop_inner_game_columns= ['W1','L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5']
# drop_location_columns= ['ATP','Series', 'Location', 'Tournament', 'Date','Comment','MaxW','MaxL','AvgW','AvgL','Wsets','Lsets']
# cleaned_df = combined_df.drop(columns= np.concatenate([drop_inner_game_columns,drop_location_columns]))
# print(f'After deleting columns:\n {cleaned_df.columns}')
cleaned_df = pd.read_csv('.\exports\concat_clean.csv')
cleaned_df.tail()

,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,...,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds,bookie_wrong,target
6081,NaN,NaN,NaN,NaN,Neuchrist M.,Bagnis F.,184.0,116.0,323.0,505.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6082,NaN,NaN,NaN,NaN,Delbonis F.,Guinard M.,215.0,267.0,270.0,201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6083,NaN,NaN,NaN,NaN,Giustino L.,Kukushkin M.,229.0,295.0,253.0,176.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6084,NaN,NaN,NaN,NaN,Collignon R.,Pacheco Mendez R.,211.0,878.0,273.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6085,NaN,NaN,NaN,NaN,Mager G.,Piraino G.,276.0,478.0,188.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# cleaned_df['Best of'] = cleaned_df['Best of'].astype(int)

### Enhance data

In [5]:
# Add rank and point difference 
rankDifference = abs(cleaned_df['WRank'] - cleaned_df['LRank'])
pointsDifference = abs(cleaned_df['WPts'] - cleaned_df['LPts'])
cleaned_df['rank_diff'] = rankDifference
cleaned_df['point_diff'] = pointsDifference
cleaned_df.head()
# cleaned_df.head

,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,...,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds,bookie_wrong,target
0,Indoor,Hard,1st Round,3.0,Basilashvili N.,Arnaboldi A.,40.0,267.0,1395.0,206.0,...,Basilashvili N.,Arnaboldi A.,40.0,267.0,1395.0,206.0,1.5,2.50,0.0,1.0
1,Indoor,Hard,1st Round,3.0,Celikbilek A.,Zuk K.,309.0,262.0,150.0,209.0,...,Celikbilek A.,Zuk K.,309.0,262.0,150.0,209.0,2.5,1.50,1.0,0.0
2,Indoor,Hard,1st Round,3.0,Ruusuvuori E.,Vesely J.,87.0,67.0,806.0,928.0,...,Ruusuvuori E.,Vesely J.,87.0,67.0,806.0,928.0,1.5,2.50,0.0,1.0
3,Indoor,Hard,1st Round,3.0,Bublik A.,Caruso S.,49.0,76.0,1090.0,858.0,...,Caruso S.,Bublik A.,76.0,49.0,858.0,1090.0,2.2,1.61,0.0,1.0
4,Indoor,Hard,1st Round,3.0,Goffin D.,Herbert P.H.,16.0,83.0,2555.0,822.0,...,Goffin D.,Herbert P.H.,16.0,83.0,2555.0,822.0,1.4,2.75,0.0,1.0


In [6]:
# Shuffle players
cleaned_df['player_1_won'] = 0

cleaned_df['swap'] = np.random.rand(len(cleaned_df)) > 0.5
cleaned_df.loc[cleaned_df['swap'], 'player_1_won'] = 1

cleaned_df['player_1'] = np.where(cleaned_df['swap'], cleaned_df['Loser'], cleaned_df['Winner'])
cleaned_df['player_2'] = np.where(cleaned_df['swap'], cleaned_df['Winner'], cleaned_df['Loser'])



In [7]:
cleaned_df['player_1_rank'] = np.where(cleaned_df['player_1_won'] == 1, cleaned_df['LRank'], cleaned_df['WRank'])
cleaned_df['player_2_rank'] = np.where(cleaned_df['player_1_won'] == 1, cleaned_df['WRank'], cleaned_df['LRank'])

cleaned_df['player_1_points'] = np.where(cleaned_df['player_1_won'] == 1, cleaned_df['LPts'], cleaned_df['WPts'])
cleaned_df['player_2_points'] = np.where(cleaned_df['player_1_won'] == 1, cleaned_df['WPts'], cleaned_df['LPts'])

In [8]:
# Shuffle odds
cleaned_df['player_1_odds'] = 0.0
cleaned_df['player_2_odds'] = 0.0
cleaned_df['player_1_odds'] = np.where(cleaned_df['swap'], cleaned_df['B365L'], cleaned_df['B365W'])
cleaned_df['player_2_odds'] = np.where(cleaned_df['swap'], cleaned_df['B365W'], cleaned_df['B365L'])
cleaned_df.head()

# Drop the 'swap' column, as it's no longer needed
cleaned_df = cleaned_df.drop(columns=['swap'])

In [9]:
# Add righthanded

In [10]:
cleaned_df.head()
cleaned_df.tail()

,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,...,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds,bookie_wrong,target
6081,NaN,NaN,NaN,NaN,Neuchrist M.,Bagnis F.,184.0,116.0,323.0,505.0,...,Bagnis F.,Neuchrist M.,116.0,184.0,505.0,323.0,1.50,2.50,NaN,NaN
6082,NaN,NaN,NaN,NaN,Delbonis F.,Guinard M.,215.0,267.0,270.0,201.0,...,Guinard M.,Delbonis F.,267.0,215.0,201.0,270.0,3.25,1.33,NaN,NaN
6083,NaN,NaN,NaN,NaN,Giustino L.,Kukushkin M.,229.0,295.0,253.0,176.0,...,Kukushkin M.,Giustino L.,295.0,229.0,176.0,253.0,2.00,1.73,NaN,NaN
6084,NaN,NaN,NaN,NaN,Collignon R.,Pacheco Mendez R.,211.0,878.0,273.0,17.0,...,Collignon R.,Pacheco Mendez R.,211.0,878.0,273.0,17.0,1.30,3.40,NaN,NaN
6085,NaN,NaN,NaN,NaN,Mager G.,Piraino G.,276.0,478.0,188.0,83.0,...,Piraino G.,Mager G.,478.0,276.0,83.0,188.0,3.00,1.36,NaN,NaN


In [11]:
# Add bookie_wrong
cleaned_df['bookie_wrong'] = cleaned_df['B365W'] > cleaned_df['B365L']
cleaned_df['bookie_wrong'] = cleaned_df['bookie_wrong'].astype(int)

### Find blanks/na

In [12]:
# find missing values in the dataframe
missing_values = cleaned_df.isna()
missing_count = cleaned_df.isna().sum()

# print the missing values
print(missing_count)

Court              27
Surface            27
Round              27
Best of            27
Winner              0
Loser               0
WRank               0
LRank               0
WPts                0
LPts                0
B365W               0
B365L               0
PSW                27
PSL                27
rank_diff           0
point_diff          0
player_1_won        0
player_1            0
player_2            0
player_1_rank       0
player_2_rank       0
player_1_points     0
player_2_points     0
player_1_odds       0
player_2_odds       0
bookie_wrong        0
target             27
dtype: int64


### Delete blanks

In [13]:
# cleaned_df = cleaned_df.dropna()

### Save cleaned df

In [14]:
# Index false means there wont be index on each line
cleaned_df.to_csv('./exports/combined_data_cleared.csv', index=False)

In [15]:
# Enhance DF with faviourite won feature (Target)

In [16]:
df = cleaned_df.copy()
df['target'] = df['B365W'] < df['B365L']
df['target'] = df['target'].astype(int)
df.tail()

,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,...,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds,bookie_wrong,target
6081,NaN,NaN,NaN,NaN,Neuchrist M.,Bagnis F.,184.0,116.0,323.0,505.0,...,Bagnis F.,Neuchrist M.,116.0,184.0,505.0,323.0,1.50,2.50,1,0
6082,NaN,NaN,NaN,NaN,Delbonis F.,Guinard M.,215.0,267.0,270.0,201.0,...,Guinard M.,Delbonis F.,267.0,215.0,201.0,270.0,3.25,1.33,0,1
6083,NaN,NaN,NaN,NaN,Giustino L.,Kukushkin M.,229.0,295.0,253.0,176.0,...,Kukushkin M.,Giustino L.,295.0,229.0,176.0,253.0,2.00,1.73,0,1
6084,NaN,NaN,NaN,NaN,Collignon R.,Pacheco Mendez R.,211.0,878.0,273.0,17.0,...,Collignon R.,Pacheco Mendez R.,211.0,878.0,273.0,17.0,1.30,3.40,0,1
6085,NaN,NaN,NaN,NaN,Mager G.,Piraino G.,276.0,478.0,188.0,83.0,...,Piraino G.,Mager G.,478.0,276.0,83.0,188.0,3.00,1.36,0,1


In [17]:
# Obsereve
print(df['player_1_won'].value_counts(dropna=False))


1    3062
0    3024
Name: player_1_won, dtype: int64


In [18]:
df.to_csv('./exports/combined_data_cleared_enhanced2.csv', index=False)

In [ ]:
### HOT ENCODE VALUES

In [ ]:
df = pd.get_dummies(df)

In [107]:
## SOME MACHINE LEARNING RANDOM FOREST CALASSIFIER

In [108]:
x = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [109]:
# cls = RandomForestClassifier(n_estimators=102,max_depth=2,random_state = 0)
# clf.fit(X_train, y_train)
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score
# y_pred = clf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# print(f'Test Accuracy: {accuracy}')
# print(f'Test Precision: {precision}')

In [110]:
X_test = X_test.reindex(columns=X_train.columns)
y_pred_proba = clf.predict_proba(X_test)
y_pred_proba

NameError: name 'clf' is not defined

In [111]:
X_test['predicted_proba'] = y_pred_proba[:, 1]

NameError: name 'y_pred_proba' is not defined

In [49]:
X_test.tail(10)

,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,...,player_1_won,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds,bookie_wrong
615,Outdoor,Hard,1st Round,3.0,Ruusuvuori E.,Kohlschreiber P.,70.0,136.0,876.0,478.0,...,0,Ruusuvuori E.,Kohlschreiber P.,70.0,136.0,876.0,478.0,1.25,4.00,0
2294,Indoor,Hard,2nd Round,3.0,Pospisil V.,Leshem E.,149.0,446.0,363.0,87.0,...,0,Pospisil V.,Leshem E.,149.0,446.0,363.0,87.0,1.33,3.40,0
2240,Outdoor,Hard,2nd Round,3.0,Nishioka Y.,Daniel T.,56.0,92.0,827.0,569.0,...,1,Daniel T.,Nishioka Y.,92.0,56.0,569.0,827.0,2.30,1.61,0
1744,Outdoor,Hard,2nd Round,3.0,Ivashka I.,Johnson S.,53.0,91.0,885.0,582.0,...,0,Ivashka I.,Johnson S.,53.0,91.0,885.0,582.0,1.33,3.40,0
2465,Indoor,Hard,2nd Round,3.0,Murray A.,Sinner J.,143.0,10.0,476.0,3395.0,...,0,Murray A.,Sinner J.,143.0,10.0,476.0,3395.0,3.20,1.36,1
718,Outdoor,Hard,1st Round,3.0,Ivashka I.,Martinez P.,115.0,100.0,673.0,787.0,...,0,Ivashka I.,Martinez P.,115.0,100.0,673.0,787.0,2.62,1.50,1
2400,Indoor,Hard,1st Round,3.0,Murray A.,Davidovich Fokina A.,48.0,31.0,975.0,1370.0,...,1,Davidovich Fokina A.,Murray A.,31.0,48.0,1370.0,975.0,2.10,1.72,0
1913,Outdoor,Hard,2nd Round,3.0,Sinner J.,Mannarino A.,12.0,70.0,2895.0,671.0,...,0,Sinner J.,Mannarino A.,12.0,70.0,2895.0,671.0,1.12,6.00,0
2132,Outdoor,Hard,1st Round,3.0,Nakashima B.,Fognini F.,83.0,31.0,885.0,1744.0,...,1,Fognini F.,Nakashima B.,31.0,83.0,1744.0,885.0,2.50,1.53,0
271,Indoor,Hard,1st Round,3.0,Mcdonald M.,Ivashka I.,56.0,48.0,1119.0,1159.0,...,1,Ivashka I.,Mcdonald M.,48.0,56.0,1159.0,1119.0,2.20,1.66,0


In [ ]:
X_test.to_csv('./exports/x_text.csv', index=False)